In [1]:
!pwd

/Users/Pavlo/PycharmProjects/ADL-Minicar-Challenge-2023/mycar/notebooks


In [2]:
import os
import time
import tensorflow as tf
import numpy as np

print(tf.__version__)

2.2.0


In [3]:
!which python

/Users/Pavlo/opt/anaconda3/envs/donkey/bin/python


In [5]:
ls ../models

arrow_sign_classifier/        pedestrian_detector_v2.0.zip
arrow_sign_classifier.tflite  pedestrian_detector_v4/
car_detector/                 stop_sign_detector/
car_detector.tflite           stop_sign_detector.tflite


In [6]:
model_path = "../models/pedestrian_detector_v4/"
tflite_model_path = "../models/pedestrian_detector_v4.tflite"

In [34]:
# Workaround for this bug https://github.com/tensorflow/tensorflow/issues/34698
bin_path = os.path.dirname(os.sys.executable)
if 'PATH' in os.environ:
    os.environ['PATH'] += ':' + bin_path
else:
    os.environ['PATH'] = bin_path
print(bin_path)
print(os.environ['PATH'])

/gpfs/space/home/pyvovar/miniconda3/envs/donkey/bin
/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/git-2.35.2-pkznwh5mpp74zq6p42b47pxpgbxxrgwk/bin:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/perl-5.34.1-4ryb2w6tlh6ha54x25kkpq7l5zjq56do/bin:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/pcre2-10.36-viek2bpe7b5ktcr3qpyl2qhipy2ybjxs/bin:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/openssh-9.0p1-xuo4l7k2kii6nqw2x2t22jqghijrsdgi/bin:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/curl-7.83.0-yzcvzf3nqftyqhrki2zketnom3lbg3kf/bin:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/openssl-1.1.1o-m6uvz6ittrnhnwd3oftfljxqwkve44j7/bin:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/libidn2-2.3.0-qmmoo5lcgqh4crq2cbbwqe26b6ujugzx/bin:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0

In [7]:
converter = tf.lite.TFLiteConverter.from_saved_model(model_path)
tflite_model = converter.convert()
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

2023-01-15 17:47:43.167352: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2023-01-15 17:47:43.250669: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f8fa13d1170 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-01-15 17:47:43.250684: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-01-15 17:47:43.707112: I tensorflow/core/grappler/devices.cc:60] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA support)
2023-01-15 17:47:43.710475: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2023-01-15 17:47:43.728032: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:797] Optimization results for grappler item: graph_to_optimize
2023-01-15 17:47:43.728435: I tensorflow/core/grapple

In [9]:
model = tf.keras.models.load_model(model_path)

input_data = np.array(np.random.random_sample([1, 70, 160, 3]), dtype=np.float32)
print(input_data.shape)

start = time.perf_counter()

output_data = model(input_data)

stop = time.perf_counter()

print(output_data)
print(f"Elapsed Time: {stop - start} ms")

(1, 70, 160, 3)
tf.Tensor([[0.9921028]], shape=(1, 1), dtype=float32)
Elapsed Time: 0.006352789999994002 ms


In [10]:
# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
print(input_data.shape)

# Inference
start = time.perf_counter()

interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# get_tensor() returns a copy of the tensor data
# use tensor() in order to get a pointer to the tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

stop = time.perf_counter()

print(output_data)
print(f"Elapsed Time: {stop - start} ms")

(1, 70, 160, 3)
[[0.9918641]]
Elapsed Time: 0.01411795000001348 ms
